# 1. 线性回归从零实现
- 了解细致的工作原理：
  - 方便自定义模型、自定义层或自定义损失函数
- 只使用张量&自动求导

In [5]:
%matplotlib inline 
import random 
import torch

## 1.1 生成数据集
- 使用低维数据
- 带有噪声
- 生成1000个样本：
  - 每个样本包含从正态分布中采样的2个特征
  - 合成数据集为一个矩阵$\mathbf{X}\in\mathbb{R}^{1000\times2}$

- 使用线性参数$\mathbf{w}=[2,-3.4]^\top$、$b=4.2$以及$\epsilon$生成数据集和标签
  - $\epsilon$服从均值为0的正态分布
  - 将标准差设为0.01

In [6]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

features, labels = synthetic_data(true_w, true_b, 1000)
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([-0.0127,  1.1306]) 
label: tensor([0.3412])


## 1.2 读取数据集
- 训练模型时需要对数据集进行遍历，每次抽取一小批量样本，并使用其更新模型
- 需要定义一个函数：
  - 可以打乱数据集中的样本
  - 并以小批量方式获取数据
- 定义该函数为`data_iter`
  - 接收参数：批大小、特征矩阵和标签向量
  - 输出：大小为`batch_size`的小批量
- `yeild`关键字
  - `yield`与`return`的区别是，`return`会返回一个最终的值，并结束函数的执行，而`yield`可以返回多个值，并保持函数的状态
  - 可以用来在一个函数中返回一个生成器对象。生成器是一种特殊的函数，它不会一次性返回所有的值，而是每次返回一个值，然后暂停执行，直到下一次请求。
  - 当一个带有yield的函数被调用时，执行会停在`yield`语句处，并在生成器被迭代时从那里继续
- 注意：
  - 下述函数在实际应用中**执行效率很低**
  - 要求所有数据加载到内存并且执行大量随机内存访问

In [7]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # Shuffle list in place, and return None.
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor (
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [8]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y) 
    break

tensor([[-0.5623, -0.6196],
        [ 1.7025, -1.1421],
        [ 1.2690, -0.1246],
        [-2.1959, -0.9279],
        [ 1.4713,  0.6683],
        [ 0.5869,  1.0196],
        [-1.5914,  0.7038],
        [-0.4241, -0.0414],
        [-0.2838,  0.1662],
        [-0.5193,  1.5036]]) 
 tensor([[ 5.1717],
        [11.4954],
        [ 7.1652],
        [ 2.9512],
        [ 4.8834],
        [ 1.9014],
        [-1.3945],
        [ 3.4953],
        [ 3.0577],
        [-1.9591]])


## 1.3 初始化模型参数
- 在使用小批量随机梯度下降优化模型参数前，需要先有一些参数
  - 通过从均值为0、标准差为0.01的正态分布中采样随机数来初始化权重，并将偏置初始化为0
- 初始化参数后，更新它们，使得其足以拟合数据
  - 每次更新都要计算损失函数关于模型参数的梯度
  - 有该梯度，则可向减少损失的方向更新每个参数

In [9]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True) 
b = torch.zeros(1, requires_grad=True)

print("Initialized parameters:\n", "\tw = ", w, "\n\tb = ", b)

Initialized parameters:
 	w =  tensor([[ 0.0039],
        [-0.0015]], requires_grad=True) 
	b =  tensor([0.], requires_grad=True)


## 1.4 定义模型及损失函数
- 定义模型：
  - 将模型的输入和参数同模型的输出关联起来
  - 计算输出：输入特征$\mathbf{X}$和模型权重$\mathbf{w}$的矩阵-向量乘法后再加上一个偏置$b$
- 定义损失函数：
  - 使用平方损失函数
  - 实现时，需要将真实值`y`的形状转换为和预测值`y_hat`的形状相同

In [10]:
def linreg(X, w, b):
    return torch.matmul(X, w) + b

def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

## 1.5 定义优化算法
- 定义函数实现小批量随机梯度下降更新
  - 从数据集中随机抽取⼀个⼩批量，然后根据参数计算损失的梯度
  - 朝着减少损失的⽅向更新参数
- 输入：模型参数集合、学习速率（`lr`）和批大小

In [11]:
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

## 1.6 训练
- 需要**重点理解**这部分代码，日后反复用到
- 每次迭代中，读取小批量训练样本，并通过模型来获得一组预测
- 计算完损失后，开始反向传播，存储每个参数的梯度
- 最后调用优化算法`sgd`来更新参数
- 概括而言，执行以下循环：
  - 初始化参数
  - 重复训练，直至完成：
    - 计算梯度：$\mathbf{g}\leftarrow \partial_{(\mathbf{w},b)}\frac{1}{\vert \mathcal{B}\vert}\sum_{i\in \mathcal{B}} l(\mathbf{x}^{(i)}, y^{(i)}, \mathbf{w}, b)$ 
    - 更新参数：$(\mathbf{w},b)\leftarrow (\mathbf{w},b)-\eta \mathbf{g}$
- 每个epoch，使用`data_iter`函数便利整个数据集，并将训练数据集中所有样本都使用一次
- 超参数：
  - `num_epochs=3`：迭代周期数
  - `lr=0.03`：学习率

In [13]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y) # X和y的小批量损失
        # 因为l形状是(batch_size,1)，⽽不是⼀个标量。l中的所有元素被加到⼀起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size) # 使⽤参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

epoch 1, loss 0.000049
epoch 2, loss 0.000049
epoch 3, loss 0.000049
w的估计误差: tensor([-7.3910e-06, -1.3399e-04], grad_fn=<SubBackward0>)
b的估计误差: tensor([-0.0005], grad_fn=<RsubBackward1>)


# 2 线性回归简洁实现
## 2.1 生成并读取数据集
- 读取数据集时:
  - 调用框架中现有的API读取
  - 将`features`和`labels`作为API的参数进行传递
  - `is_train`表示是否希望数据迭代器对象在每次迭代周期内打乱数据
- 验证`data_iter`函数是否工作正常：
  - `iter`构造Python迭代器
  - `next`从迭代器中获取第一项
- https://pytorch.org/docs/stable/data.html

In [17]:
import numpy as np
import torch
from torch.utils import data

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[ 0.4379,  0.0283],
         [-0.0777,  1.5518],
         [ 0.1473, -1.1050],
         [ 1.5232, -1.4536],
         [-1.2868, -0.9253],
         [-0.0658, -0.0709],
         [-0.2769,  0.2918],
         [-1.1718, -0.6670],
         [-0.3410,  1.9344],
         [-0.4746,  0.8176]]),
 tensor([[ 4.9633],
         [-1.2268],
         [ 8.2603],
         [12.1800],
         [ 4.7684],
         [ 4.3085],
         [ 2.6473],
         [ 4.1303],
         [-3.0531],
         [ 0.4773]])]